# Bibliotecas do projeto

In [100]:
from random import randint

# Inicializando Variáveis do projeto

1. declarar os valores iniciais
2. inicializar as variáveis compostas

In [101]:
# instancias['Configuração']
configuracao = [[1, 0], [1, 1], [0, 1]]

# janela Disponibilidade = Disponibilidade total de tempo de uma equipe
janela_disponibilidade = [30, 40]

# Janela Final = capacidade de processamento de cada maquina
janela_final = [27, 19, 25, 35]

# Janela Inicial = tempo em que cada maquina deveria começar
janela_inicial = [3, 3, 4, 7]

# instancias['Processamento']
# processamento = atividade x maquina
processamentos = [ [3, 3, 4, 1],
                   [2, 3, 1, 1],
                   [1, 1, 1, 2] ]

# ----------------------------------------------------
# criacao das equipes (indexadas em 1)

numero_equipes = len(configuracao[0])
equipes = dict()

for numero in range(numero_equipes):
  nome_equipe = 'eq' + str(numero+1)
  equipes[nome_equipe] = dict()
  equipes[nome_equipe]['janela final local'] = [ ]
  equipes[nome_equipe]['janela total'] = [ ]
  equipes[nome_equipe]['janela inicial'] = [ ]
  equipes[nome_equipe]['disponibilidade'] = janela_disponibilidade[numero]
  equipes[nome_equipe]['disponibilidade total'] = janela_disponibilidade[numero]
  equipes[nome_equipe]['tempo'] = 0
  
  for i in range(len(janela_final)):
    valor = janela_final[i]
    valor_inicial = janela_inicial[i]
    equipes[nome_equipe]['janela final local'].append(valor)
    equipes[nome_equipe]['janela total'].append(valor)
    equipes[nome_equipe]['janela inicial'].append(valor_inicial)

  equipes[nome_equipe]['maquinas'] = [ ]

# Definindo os filtros

Filtros serão funções para buscarmos equipes que podem receber alguma determina máquina baseado em alguns critétios

In [102]:
def filtraAtivos(vetor_binario):
  ativos = [ ]

  for i in range( len(vetor_binario) ):
    if vetor_binario[i] : ativos.append(i)
  
  return ativos

In [103]:
def filtraAtivosPorJanelaFinal(maquina, ativos, equipes):

  numero_da_maquina = maquina['maquina de origem']
  janela = maquina['janela']
  ativos_temporarios = [ ]

  for i in range( len(ativos) ):
    ativo = ativos[i]
    nome_equipe = 'eq' + str(ativo + 1)

    # DEBUG
    # print(f'{nome_equipe}')
    
    equipe = equipes[nome_equipe]
    janela_na_equipe_para_maquina = equipe['janela final local'][numero_da_maquina]

    # DEBUG
    # print(f'espaço: {janela_na_equipe_para_maquina} || custo {janela}')

    if janela_na_equipe_para_maquina >= janela:
      ativos_temporarios.append(ativo)

  return ativos_temporarios

In [104]:
def filtraAtivosPorDisponibilidade(maquina, ativos, equipes):
  
  numero_da_maquina = maquina['maquina de origem']
  janela = maquina['janela'] 
  ativos_temporarios = [ ]

  for i in range( len(ativos) ):
    ativo = ativos[i]
    nome_equipe = 'eq' + str(ativo + 1)
    equipe = equipes[nome_equipe]
    disponibilidade = equipe['disponibilidade']
    
    # DEBUG
    # print(f'janela equipe {nome_equipe} para maquina {numero_da_maquina}: {janela_na_equipe_para_maquina}')
    # print(f'tempo de janela: {janela}')

    if disponibilidade >= janela:
      ativos_temporarios.append(ativo)

  return ativos_temporarios

# Funções de distribuição entre equipes e ativos

In [105]:
def processaJanelaInicial(maquina, equipe):

  numero_da_maquina = maquina['maquina de origem']
  janela = maquina['janela'] 
    
  horario_autorizado = equipe['janela inicial'][numero_da_maquina]

  # precisa esperar ?
  tempo = max(equipe['tempo'], horario_autorizado) + janela
  
  equipe['tempo'] = tempo  

In [106]:
def distribuiParaAtivos(processamento, ativos, equipes, indice_atividade) :

  for maquina_index in range( len(processamento) ):

    maquina = {
      'janela': processamento[maquina_index],
      'indice da atividade': indice_atividade,
      'maquina de origem': maquina_index
    }
    
    # ativos 1 = filtrados pela janela final
    ativos1 = filtraAtivosPorJanelaFinal(maquina, ativos, equipes)

    # ativos 2 = filtrados pela disponibilidade da equipe
    ativos2 = filtraAtivosPorDisponibilidade(maquina, ativos1, equipes)
   
    # DEBUG
    # print(f'ATIVOS = {ativos1} ')
    # print(f'POR FINAL = {ativos2} ')
    
    # resgato a posicao em cima do ultimo filtro
    pos = randint(0, len(ativos2)-1 )
   
    ## monto o nome da equipe em funcao dos filtrados
    nome_equipe = 'eq' + str(ativos2[pos] + 1)
    equipe = equipes[nome_equipe]
   
    #processando janela inicial
    # ainda não valida se o tempo total é menor que a janela final
    ativos2 = processaJanelaInicial(maquina, equipe)

    equipe['janela final local'][maquina_index] -= maquina['janela']
    equipe['disponibilidade'] -= maquina['janela']
    
    # DEBUG
    # print(f'A equipe {nome_equipe} recebeu na maquina {maquina_index} menos {maquina["janela"]}')

    equipe['maquinas'].append(maquina)

In [107]:
def divideMatriz(configuracao, processamento, equipes):

  for linha in range( len(configuracao) ) :
    ativos = filtraAtivos(configuracao[linha])
    distribuiParaAtivos(processamento[linha], ativos, equipes, linha)   

# Cálculos em cima da distribuição final

In [108]:
def tempo_da_equipe_maior_tempo_gasto(equipes):

  maior_tempo = -1

  for i in range(len(equipes)) :
    nome_equipe = 'eq' + str(i + 1)
    equipe = equipes[nome_equipe]

    tempo_gasto = equipe['tempo']

    if tempo_gasto > maior_tempo:
      maior_tempo = tempo_gasto

  return maior_tempo

# Funções de Impressão

In [109]:
def printEquipes():
  for key in equipes.keys():
    print(f'# {key}')
    print(f'janela final local final : {equipes[key]["janela final local"]} ')

    for maquina in equipes[key]['maquinas']:
      print(f'{maquina}')

# Parte principal

In [110]:
divideMatriz(configuracao, processamentos, equipes)

for key in equipes.keys():
  print(f'# {key}')
  print(f'disponibilidade total : {equipes[key]["disponibilidade total"]} ')
  print(f'disponibilidade resultante : {equipes[key]["disponibilidade"]} ')
  print(f'tempo gasto da equipe total : {equipes[key]["tempo"]}')
  print(f'janela final local final : {equipes[key]["janela final local"]} ')
  print(f'janela total : {equipes[key]["janela total"]} ')
  print(f'consumo da janela inicial : {equipes[key]["janela inicial"]} ')

  for maquina in equipes[key]['maquinas']:
    print(f'{maquina}')

tempo_maior = tempo_da_equipe_maior_tempo_gasto(equipes)
print(f'MAIOR TEMPO GASTO EM UMA EQUIPE: {tempo_maior}')

ATIVOS = [0] 
POR FINAL = [0] 
ATIVOS = [0] 
POR FINAL = [0] 
ATIVOS = [0] 
POR FINAL = [0] 
ATIVOS = [0] 
POR FINAL = [0] 
ATIVOS = [0, 1] 
POR FINAL = [0, 1] 
ATIVOS = [0, 1] 
POR FINAL = [0, 1] 
ATIVOS = [0, 1] 
POR FINAL = [0, 1] 
ATIVOS = [0, 1] 
POR FINAL = [0, 1] 
ATIVOS = [1] 
POR FINAL = [1] 
ATIVOS = [1] 
POR FINAL = [1] 
ATIVOS = [1] 
POR FINAL = [1] 
ATIVOS = [1] 
POR FINAL = [1] 
# eq1
disponibilidade total : 30 
disponibilidade resultante : 19 
tempo gasto da equipe total : 14
janela final local final : [24, 16, 21, 34] 
janela total : [27, 19, 25, 35] 
consumo da janela inicial : [3, 3, 4, 7] 
{'janela': 3, 'indice da atividade': 0, 'maquina de origem': 0}
{'janela': 3, 'indice da atividade': 0, 'maquina de origem': 1}
{'janela': 4, 'indice da atividade': 0, 'maquina de origem': 2}
{'janela': 1, 'indice da atividade': 0, 'maquina de origem': 3}
# eq2
disponibilidade total : 40 
disponibilidade resultante : 28 
tempo gasto da equipe total : 15
janela final local final : [